In [1]:
from dataclasses import make_dataclass, field
from functools import partial, reduce

import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import boschloo_exact, fisher_exact, barnard_exact

In [2]:
FUNCTIONS = [boschloo_exact, fisher_exact, barnard_exact]

In [3]:
def get_binom_samples(n, p, size=1000):
    tables = stats.binom.rvs(n, p, size=size)
    return tables

ResPvalues = make_dataclass('ResPvalues',
                   [('boschloo_exact', float, field(default=0)),
                    ('barnard_exact', float, field(default=0)),
                    ('fisher_exact', float, field(default=0)),],)

def get_pvalues(table, alt="two-sided"):
    res = ResPvalues()
    for fn in FUNCTIONS:
        if fn.__name__ == "fisher_exact":
            _, res.fisher_exact = fisher_exact(table, alternative=alt)
        
        else:
            setattr(res, fn.__name__, fn(table, alternative=alt).pvalue)
    return res


def count_treshold(map_obj, alpha):
    def cond(res: ResPvalues):
        """
        Condition if we accept H_0 or not
        """
        boschloo = 1 if res.boschloo_exact > alpha else 0
        barnard = 1 if res.barnard_exact > alpha else 0
        fisher = 1 if res.fisher_exact > alpha else 0
        return np.array([boschloo, barnard, fisher])
    
    return list(
        reduce(lambda x, y: x + y, map(cond, map_obj), np.array([0,0,0]))
    )
            

Let's do this in parallele now

In [4]:
from power_comparison_with_processus import count_rejection_hypothesis_with_workers
count_rejection_hypothesis_with_workers()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 20.65it/s]

[(0.3328284170851499, 0.37983080474083114, 0.4029781876559133), (0.0770889841659394, 0.08140662639496644, 0.10925007790438504), (0.04070892515274173, 0.04633099139466005, 0.06142495023503493), (0.005852733258053538, 0.008223035085892233, 0.009592280122885434), (0.0015043443859467467, 0.0025184972541036446, 0.00255127504381859), (0.04070892515274173, 0.04633099139466005, 0.06142495023503493), (0.23274656531266344, 0.22796585327577495, 0.28423703690529284), (0.2845613884183985, 0.37973882448930674, 0.3455364191791259), (0.026838037432909127, 0.04521028814895972, 0.04112758275423647), (0.3328284170851499, 0.37983080474083114, 0.4029781876559133), (0.4551557946673382, 0.45504085915226805, 0.5372073145913876), (0.130677515880254, 0.1611190736047562, 0.17638438259140393), (0.47067138352390964, 0.46279508311437023, 0.548239272941272), (0.8467880965733328, 1.0, 0.8862454770455244), (0.14241818055834357, 0.19788572579621097, 0.1854118737244251), (0.1353267298560647, 0.19788572579621092, 0.18193